In [ ]:
!pip install timm -q

In [ ]:
import os
import time
import numpy as np
import pandas as pd

# image manipulation
import cv2
import PIL
from PIL import Image

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# helpers
from tqdm import tqdm
import time
import copy
import gc
from enum import Enum

from sklearn.metrics import roc_auc_score, precision_recall_curve

import timm


# for cnn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, SGD
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split, TensorDataset, Dataset, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torchvision import models
from torchmetrics.classification import BinaryF1Score, BinaryPrecision, BinaryRecall, BinaryAccuracy, BinaryROC, BinaryAUROC
from torchvision import transforms
import albumentations as A

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(42)

In [ ]:
class RSNAMamographyDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.df = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    

    def __getitem__(self, ind):
        
        img_path = f"{self.img_dir}/{self.df.iloc[ind].patient_id}_{self.df.iloc[ind].image_id}.png"
        img = Image.open(img_path).convert('RGB')
        
        label = self.df.iloc[ind].cancer
        # there is no need to normalize data, it has already been normalized
        if self.transform:
            img = self.transform(img).to(torch.float32) 
        else:
            default_transform = transforms.Compose([transforms.ToTensor()])
            img = default_transform(img).to(torch.float32)
            
        #sample = {"image" : img, "label": label}
        
        label = label.to(torch.long)
        
        return img, label

In [ ]:

class Dataset:
    def __init__(self, df, transform):
        self.df = df.copy()
        self.transform = transform
     
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        patient_id = self.df.loc[idx, 'patient_id']
        image_id = self.df.loc[idx, 'image_id']
        # Target
        target = self.df.loc[idx, 'cancer'] 
        
        # Get and preprocess images
        #dcm_path = '/kaggle/input/rsna-breast-cancer-detection/train_images/'
        #image = dicom.dcmread(image_path)
        
        #png_path = '/kaggle/input/rsna-png-images-same-format-as-original/output/rsna_pngs/train_images'
        #png_path = '/kaggle/input/png-cutted-images/output/png_cutted_images' # my work
        png_path = '/kaggle/input/preprocessed-images/train_images' # préprocessed images by vladimir
        
        # Image path
        #image_dcm_path =  os.path.join(png_path, patient_id.astype(str), image_id.astype(str)+'.dcm')
        image_png_path =  os.path.join(png_path, patient_id.astype(str), image_id.astype(str)+'.png')

        # convert image to RGB
        image = Image.open(image_png_path).convert('RGB')
        
        # Check if image is RGB 
        #print('image shape = ', np.array(image).shape)
        
        image = image.resize((1024,912)) # 512,512 ??
        
        # Apply transformers on images
        if (target == 1) and self.transform:
            image = self.transform(image).to(torch.float32)
        else :
            default_transform = transforms.Compose([transforms.ToTensor()])
            image = default_transform(image).to(torch.float32)
            #image = image.to(torch.float32)
                    
        # Convert to tensors
        #image = torch.tensor(image, dtype=torch.float32)
        
        target = torch.tensor(target, dtype=torch.long)
        
        return image, target

In [ ]:
# Resnet and Efficientnet
class Models(nn.Module):
    def __init__(self):
        super(Models, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        # Resnet
        self.resnet = timm.create_model('resnet18', pretrained=True, in_chans=3)
        n_features_resnet = self.resnet.fc.out_features
        
        self.classifier_layer_resnet = nn.Sequential(
            nn.Linear(n_features_resnet , 256),
            nn.Dropout(0.8), #0.3
            nn.Linear(256 , 1) #NUM_CLASSES = 1
        )
        
        # Efficientnet
        self.efficientnet = timm.create_model('efficientnet_b4', pretrained=True, in_chans=3)
        in_features_efficientnet = self.efficientnet.classifier.in_features
        
        self.classifier_layer_efficientnet = nn.Sequential(
            nn.Linear(in_features_efficientnet , 256),
            nn.Dropout(0.3),
            nn.Linear(256 , 1)
        )
    
    #if resenet
    #def forward(self, xb):        
     #   xb = self.resnet(xb)
     #   xb = self.classifier_layer_resnet(xb)
     #   return xb
    
    #if efficientnet
    def forward(self, xb):        
        xb = self.efficientnet(xb)
        xb = self.classifier_layer_efficientnet(xb)
        return xb
    
        #return torch.sigmoid(xb)
    

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        self.network = timm.create_model('resnet18', pretrained=True, in_chans=3)
        n_features = self.network.fc.out_features
        
        # add additional layer that maps 2048 extracted features from resnet to 1 feature 
        #determining the class
        self.classifier_layer = nn.Sequential(
            nn.Linear(n_features , 256),
            nn.Dropout(0.8),
            nn.Linear(256 , 1)
        )
    
    def forward(self, xb):        
        xb = self.network(xb)
        xb = self.classifier_layer(xb)
        return torch.sigmoid(xb)

In [ ]:
# Only resnet
class CNN_RESNET(nn.Module):
    def __init__(self):
        super(CNN_RESNET, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        self.resnet = timm.create_model('resnet18', pretrained=True, in_chans=3) #resnet18
        n_features = self.resnet.fc.out_features
        
        # add additional layer that maps 2048 extracted features from resnet to 1 feature 
        #determining the class
        self.classifier_layer = nn.Sequential(
            nn.Linear(n_features , 256),
            nn.Dropout(0.8), #0.3
            nn.Linear(256 , 1)
        )
    
    def forward(self, xb):        
        xb = self.resnet(xb)
        xb = self.classifier_layer(xb)
        return xb
        #return torch.sigmoid(xb)
    

In [ ]:

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (
            self.__class__.__name__
            + "("
            + "p="
            + "{:.4f}".format(self.p.data.tolist()[0])
            + ", "
            + "eps="
            + str(self.eps)
            + ")"
        )
    
class MammoModel(nn.Module):
    def __init__(self, name, *, pretrained=False, in_chans=3, p=3, p_trainable=False, eps=1e-6):
        super().__init__()
        model = timm.create_model(name, pretrained=pretrained, in_chans=in_chans)
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        
        self.fc = nn.Linear(n_features, 1) # cancer
        self.model = model

        self.pool = nn.Sequential(
            GeM(p=p, eps=eps, p_trainable=p_trainable),
            nn.Flatten())
    
    def forward(self, x):
        x = self.model(x)
        #x = self.model.forward_features(x)
        #x = self.pool(x)
        logits = self.fc(x)
        return logits

In [ ]:
''' Model (we need to improve our model''' 
class BreastCancerModel(nn.Module):
    def __init__(self, Config):
        super().__init__()
        # efficientnet
        self.efficientnet = timm.create_model('efficientnet_b4', pretrained=True,
                                             in_chans=3)
        in_features = self.efficientnet.classifier.in_features
        self.efficientnet.classifier = nn.Linear(in_features, Config.NUM_CLASSES)  
        
        
        # Resnet
        self.resnet = timm.create_model('resnet50', pretrained=True, in_chans=3)
        self.resnet.fc = nn.Linear(2048, Config.NUM_CLASSES)
    
        
    def forward(self, image):
        output = self.efficientnet(image)
        #output = self.resnet(image)
    
        return output
    

In [ ]:
def BCELoss_class_weighted(weights):
    """
    weights[0] is weight for class 0 (negative class)
    weights[1] is weight for class 1 (positive class)
    """
    def loss(y_pred, target):
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7) # for numerical stability
        bce = - weights[1] * target * torch.log(y_pred) - (1 - target) * weights[0] * torch.log(1 - y_pred)
        return torch.mean(bce)

    return loss

In [ ]:
# create class for earlystopping
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_loss = np.inf

    def early_stop(self, loss):
        if loss <= self.min_loss:
            self.min_loss = loss
            self.counter = 0
        elif loss > (self.min_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def find_optimal_threshold(fpr, tpr, thresholds): # based on ROC curve
    J = tpr - fpr
    index = np.argmax(J)
    OptThreshold = thresholds[index]
    return OptThreshold
    
    

In [ ]:
'''Training function'''
def train(data_loader, model, optimizer, device, scheduler=None):
    # Set model in training mode
    model.train()
    
    # Metrics
    metricf1 = BinaryF1Score()
    precision = BinaryPrecision()
    recall = BinaryRecall()
    accuracy = BinaryAccuracy()
    roc = BinaryROC()
    auc = BinaryAUROC()
    
    train_metrics = {'loss' : [], 'acc' : [], 'f1': [], 'precision': [], 'recall': [], 'auc': []}
    
    # Initial threshold
    threshold = 0.5
        

    # DataLoader
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    running_loss = 0
    correct = 0
    total = 0
    all_outputs = torch.Tensor([])
    all_labels = torch.Tensor([]) # needed to calculate auc score ??
   
    for batch_idx, data in enumerate(tk0):
        images = data[0]
        targets = data[1]
        
        targets = torch.unsqueeze(targets, 1)
        
        images = images.to(device).float() 
        targets = targets.to(device).float() # long
        
        #print('targets shape = ', targets.shape)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
        
            outputs = model(images)#.squeeze()
            #print('outputs shape = ', outputs.shape)

            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            # all outputs
            all_outputs = torch.cat((all_outputs, outputs.to('cpu')))
            
            # all labels
            all_labels = torch.cat((all_labels, targets.to('cpu')))
            

            # Correct classifications
            probs = outputs#.sigmoid() # in this case we don't need sigmoid because we have it in the 
            # model class !!
            predicted_vals = probs > threshold
            correct += torch.sum(predicted_vals == targets.data)


            # Total classifications
            total += targets.size(0)
            
        #auc score
        #auc_score = roc_auc_score(all_labels.detach().numpy(), all_outputs .detach().numpy())
        
        # running loss    
        running_loss += loss.item()

        # collect any unused memmory
        #gc.collect()
        #torch.cuda.empty_cache()

    #Accuracy
    accu = correct/total
    
    # Average training loss (epoch loss)
    train_loss = running_loss / len(data_loader)
    
    #auc score
    auc_score = roc_auc_score(all_labels.detach().numpy(), all_outputs.detach().numpy())
    
    
    # ROC
    print('all_labels type', all_labels.dtype)
    fpr, tpr, thresholds = roc(all_outputs, all_labels.long())
    
    
    # Find optimal threshold
    OptThreshold = find_optimal_threshold(fpr, tpr, thresholds)
    
    
    print(f'New threshold is {OptThreshold}')
    
    # Calculate metrics using optimized threshold
    # f1 score
    f1_measure = metricf1(all_outputs > OptThreshold, all_labels)
    
    '''
    # need to be runned ??
    # find True positive rate and False postive rate for ROC curve
    fpr, tpr, thresholds = roc(all_outputs, all_labels)
    epoch_auc = auc(all_outputs, all_labels)

    # Find new threshold
    # we need a function here to find the optimal threshold

    # Calculate metrics using the optimized threshold
    f1_measure = metricf1(all_outputs > threshold, all_labels)
    epoch_acc = accuracy(all_outputs > threshold, all_labels)
    epoch_precision = precision(all_outputs > threshold, all_labels)
    epoch_recall = recall(all_outputs > threshold, all_labels)

    # Save the metrics
    scheduler.step()
    train_metrics['loss'].append(train_loss)
    train_metrics['acc'].append(epoch_acc)
    train_metrics['f1'].append(f1_measure)
    train_metrics['precision'].append(epoch_precision)
    train_metrics['recall'].append(epoch_recall)
    train_metrics['auc'].append(epoch_auc)
    
    tr_loss, tr_acc, tr_f1, tr_prec, tr_rec, tr_auc = train_metrics['loss'], 
                                                train_metrics['acc'],  train_metrics['f1'],
                                                train_metrics['precision'], 
                                                train_metrics['recall'], 
                                                train_metrics['auc']
    
    #tr_loss, tr_acc, tr_f1, tr_prec, tr_rec, tr_auc = train_metrics['loss'][-1], 
    #                                            train_metrics['acc'][-1],  train_metrics['f1'][-1],
    #                                            train_metrics['precision'][-1], 
    #                                            train_metrics['recall'][-1], 
    #                                            train_metrics['auc'][-1]
                                                
    print(f'Train Loss: {tr_loss:.4f}, Train Acc: {tr_acc:.4f}, 
          Train f1: {tr_f1:.4f}, Train Precision: {tr_prec:.4f}, 
          Train Recall: {tr_rec:.4f}, Train AUC: {tr_auc:.4f}')

    '''
   
    print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))
    print('auc_score : %.3f'%(auc_score))
    print('f1_measure : %.3f'%(f1_measure))
  
    
   


In [ ]:
'''Evaluation function'''

def evaluation(data_loader, model, device):
    model.eval()
    
    # Metrics
    metricf1 = BinaryF1Score()
    precision = BinaryPrecision()
    recall = BinaryRecall()
    accuracy = BinaryAccuracy()
    roc = BinaryROC()
    auc = BinaryAUROC()
    
    
    predictions = []
    tar = []
    running_loss = 0
    correct = 0
    total = 0
    
    threshold = 0.5
    
    all_outputs = torch.Tensor([])
    all_labels = torch.Tensor([]) # needed to calculate auc score ??
    
    for batch_idx, data in enumerate(data_loader):
            images = data[0]
            targets = data[1]
            
            #targets = torch.unsqueeze(targets.to(torch.float), 1) #float32
            targets = torch.unsqueeze(targets, 1)
        
            images = images.to(device, dtype=torch.float) 
            targets = targets.to(device, dtype=torch.float) # long ??
       
            with torch.no_grad():
                outputs = model(images)#.squeeze()
                loss = criterion(outputs, targets) 
                
            # all outputs
            all_outputs = torch.cat((all_outputs, outputs.to('cpu')))
            
            # all labels
            all_labels = torch.cat((all_labels, targets.to('cpu')))    
            
            running_loss += loss.item()
            
            # Correct classifications (for accuracy calculation)
            probs = outputs#.sigmoid()
            predicted_vals = probs > threshold
            correct += torch.sum(predicted_vals == targets.data)
            # Total classifications
            total += targets.size(0)
            
    
            # Predictions
            predictions.append(predicted_vals.cpu().numpy())
            
            # Targets
            tar.append(targets.cpu().numpy())
            
            # collect any unused memmory
            #gc.collect()
            #torch.cuda.empty_cache()
     
    # Loss 
    eval_loss = running_loss / len(data_loader)
    # Accuracy
    eval_accu = correct/total
    print('Eval Loss: %.3f | Accuracy: %.3f'%(eval_loss,eval_accu))
    
    #auc score
    eval_auc_score = roc_auc_score(all_labels.detach().numpy(), all_outputs .detach().numpy())
    
    # ROC
    print('type of labels', all_labels.dtype)
    fpr, tpr, thresholds = roc(all_outputs, all_labels.long())
    
    # Find optimal threshold
    OptThreshold = find_optimal_threshold(fpr, tpr, thresholds)
    
    # Calculate metrics using optimized threshold
    # f1 score
    eval_f1_measure = metricf1(all_outputs > OptThreshold, all_labels)
    
    
    # Predictions
    predictions = np.concatenate(predictions) # this line convert the list 
      # of lists into a 1d array.
        
    # Targets    
    tar = np.concatenate(tar)
    
    print('Eval Loss: %.3f | Eval Accuracy: %.3f'%(eval_loss,eval_accu))
    print('Eval auc_score : %.3f'%(eval_auc_score))
    print('Eval_f1_measure : %.3f'%(eval_f1_measure))

    return predictions, tar, OptThreshold     
    

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    
    train_metrics = {'loss' : []}
    val_metrics = {'loss' : []}
    
    print('Starting training...')
    print('-' * 20)
    for epoch in range(num_epochs):
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training model
            else:
                model.eval()   # Set model to evaluate model

            running_loss = 0.0
            
            # Iterate over data.
            print(f'{phase} for epoch {epoch + 1}')
            for inputs, labels in tqdm(dataloaders[phase]):
                print('labels shape before unsqueeze : ', labels.shape)
                labels = torch.unsqueeze(labels.to(torch.long), 1) #float32
                print('labels shape after unsqueeze : ', labels.shape)
            
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    print('outputs shape = ', outputs.shape)
                    preds = (outputs > 0.5).double()
                                       
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        

                # statistics
                running_loss += loss.item()

                # collect any unused memmory
                gc.collect()
                torch.cuda.empty_cache()
            
            # statistics
            epoch_loss = running_loss / len(dataloaders[phase])
            
            # save all of the statistics for latter analysis
            if phase == 'train':
                scheduler.step()
                train_metrics['loss'].append(epoch_loss)
                
            else:
                val_metrics['loss'].append(epoch_loss)
                
                    
        # cant be formated in string
        tr_loss = train_metrics['loss'][-1]
        val_loss = val_metrics['loss'][-1]
        
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Train Loss: {tr_loss:.4f}')
        print(f'Valitadion Loss: {val_loss:.4f}')
        
        if earlystoper.early_stop(val_loss):
            break
        
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    
    return train_metrics, val_metrics

In [ ]:
''' Probabilistic F1 score'''
def pfbeta(labels, predictions, beta):
    y_true_count = 0
    ctp = 0
    cfp = 0
    
    
    for idx in range(len(labels)):  
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [ ]:
train_csv = '/kaggle/input/rsna-breast-cancer-detection/train.csv'
imgs_dir = '/kaggle/input/rsnamamorgaphybreastcancerrecognition512x512'

'''
augmentator = transforms.Compose([
    # input for augmentator is always PIL image
    # transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(5),
    transforms.ToTensor(), # return it as a tensor and transforms it to [0, 1]
])
'''
transform = transforms.Compose([
    transforms.ToTensor()])

'''
augmentator = transforms.Compose([
    # input for augmentator is always PIL image
    # transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomAffine(degrees=(0, 180), scale=(0.8, 1.2)),
    transforms.ElasticTransform(),
    transforms.ToTensor(), # return it as a tensor and transforms it to [0, 1]
])

'''

augmentator = transforms.Compose([
    # input for augmentator is always PIL image
    # transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    #transforms.RandomPerspective(),
    #transforms.RandomRotation((0, 90)),
    #transforms.RandomAutocontrast(),
    #transforms.RandomAffine(degrees=(0, 180), scale=(0.8, 1.2)),
    #transforms.ElasticTransform(),
    transforms.ToTensor(), # return it as a tensor and transforms it to [0, 1]
])


#dataset = RSNAMamographyDataset(train_csv, imgs_dir, augmentator)
dfx = pd.read_csv(train_csv)
dataset = Dataset(dfx, transform=augmentator)


In [ ]:
val_pct = 0.1
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
''' WeightedRandomSampler for imbalanced data'''

'''
dftrain = pd.read_csv(train_csv)


print("Class counting...")
labels = dftrain['cancer'].values
class_sample_count = np.array([len(np.where(labels == l)[0]) for l in np.unique(labels)])

print(class_sample_count)

print("Adding weights to each training sample...")
# class_sample_count[1] *= 5 # ??
class_weights = 1. / class_sample_count
sample_weights = []
for _, label in tqdm(train_dataset):
    sample_weights.append(class_weights[label])
    
    
# the trouble with this aproach is that it now has to load all images one by one and label them
# but it saves RAM memory in training process    
    
sample_weights = np.array(sample_weights)
sample_weights = torch.from_numpy(sample_weights)

'''

In [ ]:
'''
weighted_random_sampler = WeightedRandomSampler(sample_weights, len(sample_weights),
                                               replacement=True)
weighted_random_sampler

'''

In [ ]:
batch_size =  32 #32

# Applying sampler just to train dataset, not for validation, since the validation dataset 
# should be an imitation of real Dataset

#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle = False, num_workers = 2, 
#                              pin_memory = True, sampler = weighted_random_sampler) # using our sampler

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers = 2, 
                              pin_memory = True) # random sampler

val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False, pin_memory = True)
# If you use shuffle = True the DataLoader will initialize a RandomSampler for you, 
# otherwise it’ll use SequentialSampler.

In [ ]:
dataloaders = {'train' : train_dataloader, 'val' : val_dataloader}
dataset_sizes = {'train': train_size, 'val' : val_size}

In [ ]:
''' Weighted loss for imbalanced data '''

# Number of samples for each class
samples_classes = dfx['cancer'].value_counts()
print(samples_classes)

#pos_weight = 53548/1158
pos_weight = torch.tensor([46]).cuda()

#criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        self.network = timm.create_model('resnet18', pretrained=True, in_chans=3)
        n_features = self.network.fc.out_features
        
        # add additional layer that maps 2048 extracted features from resnet to 1 feature 
        #determining the class
        self.classifier_layer = nn.Sequential(
            nn.Linear(n_features , 256),
            nn.Dropout(0.3),
            nn.Linear(256 , 1)
        )
    
    def forward(self, xb):        
        xb = self.network(xb)
        xb = self.classifier_layer(xb)
        return xb

In [ ]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
#model = MammoModel('efficientnet_v2_s', pretrained=True) #efficientnet_b5
model = CNN()
#model = BreastCancerModel(Config=Config)
model.to(device)

# Define the criterion (loss)
#w_pos = 3
#w_neg = 1
#criterion = BCELoss_class_weighted(weights = [w_neg, w_pos])

# or
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2.56e-05)

# Define the metric
metric = BinaryF1Score().to(device)


earlystoper = EarlyStopper(patience = 3)

# Scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


# Model path
#model_path = f"model_{fold}.bin"
model_path = "model.bin"
    
# Run
best_score = 0
for epoch in range(5):
    train(train_dataloader, model, optimizer, device, scheduler)
    predictions, targets, OptThreshold = evaluation(val_dataloader, model, device)

    print('epoch = ', epoch)

    F1_score = pfbeta(targets, predictions, beta=1)

    if F1_score > best_score:
        best_score = F1_score
        
        PATH = "model.pt"
        checkpoint = torch.save({
            'model_state_dict': model.state_dict(),
            'threshold' : OptThreshold
            }, PATH)
        
    print('f1_score = ', best_score)     

print('best_score_ever = ', best_score) 


In [ ]:
# In this version, we used :
# - only augmentations,
# - preproccessed images from vladimir, notebook : https://www.kaggle.com/code/rimzakhama/cut-off-empty-space-from-images/notebook
# 